<a href="https://colab.research.google.com/github/dipanshuhaldar/binary_classification_pytorch/blob/master/pytorch_mlp_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* We will use the [**Ionosphere binary (two class) classification dataset**](https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/) to demonstrate an MLP for binary classification.
* This dataset involves predicting whether there is a structure in the atmosphere or not given radar returns.

In [2]:
'''
set working directory
look in the files in the working directory
'''
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/ionosphere')
os.listdir('/content/drive/My Drive/ionosphere')

['Ionosphere.csv', 'pytorch_mlp_binary_classification.ipynb']

In [0]:
#requirements
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score 

In [0]:
#requirements(PyTorch)
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [0]:
#df = read_csv('Ionosphere.csv')
#df.head()
#X = df.values[:, df.columns != 'Class']
#X[0:2, :]

**Loading and Data Preparation**

In [0]:
#dataset definition
class CSVDataset(Dataset):
  #load the data set
  def __init__(self, path):

    #load the csvfile as a dataframe
    df = read_csv(path)
    #store the features and the target
    self.X = df.values[:, df.columns != 'Class']#desection on column name, here Class
    self.y = df.values[;, 'Class']#selection based on column name, here Class
    #ensure input data is floats
    self.X = self.X.astype('float32')
    #label encode target and ensure the values are floats
    self.y = LabelEncoder().fit_transform(self.y)
    self.y = self.y.astype('float32')
    self.y = self.y.reshape(len(self.y), 1)

    #number of rows in the dataset
    def __len__(self):
      return len(self.X)

    #get a row by index
    def __getitem__(self, idx):
      return [self.X[idx], self.y[idx]]

    #get indexes for train and test rows
        